In [1]:
import numpy as np
import sys
from pathlib import Path
import json

np.random.seed(32)

In [2]:
project_root = Path.cwd().parent
sys.path.append(str(project_root))

from BayesianOptimization.BO_functions import probability_of_improvement, expected_improvement, GP_UCB_original, squared_exponential_kernel, fit_predictive_GP, optimize_GP_hyperparams

data = np.load('../game_results/game_results_humans.npz')
xyz = data['xyz']
params = data['params']
meta = data['meta']

/Users/Liv/Library/CloudStorage/OneDrive-Personligt/KID/4. semester/Active Machine Learning and Agency (02463)/Active ML Python/Group-24-Bayesian-Optimization/.venv/lib/python3.12/site-packages/torch/__init__.py:1305: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/tensor/python_tensor.cpp:436.)
  _C._set_default_tensor_type(t)


### Functions

In [3]:
# Ground truth model in numpy
def gmm_model(params, x, y):
    # params: (N, 25), x/y: (N, 20)
    g = params[:24].reshape(6, 4)
    w, mx, my, s = g[:, 0], g[:,1], g[:, 2], g[:, 3]
    expo = -((x[...,None] - mx[None])**2 + (y[...,None] - my[None])**2) / s[None]
    res = np.sum(w[None] * np.exp(expo), axis=-1)
    
    return res * params[24]

# Softmax function for converting log probabilities to probabilities
def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

# frim x,y coordinates to flat index in 100x100 grid
def xy_to_flat_index(x, y):
    xi = int(np.clip(np.floor(x * 100), 0, 99))
    yi = int(np.clip(np.floor(y * 100), 0, 99))
    return yi * 100 + xi

## Main GP–Acquisition Evaluation Loop

In [4]:
X = np.linspace(0,1,100,endpoint=False)
xx, yy = np.meshgrid(X,X)
Xtest = np.column_stack([xx.ravel(), yy.ravel()])

def f_objective(point):
    x,y=point
    x = min(99,int(x*100))
    y = min(99, int(y*100))
    return objective[x,y]

prior_mean = 0 
prior_std = 5
xi = 0.0001        # for EI and PI
kappa = 0.5      # for UCB
    
results = []

for i, person in enumerate(xyz[:20]):
    # First 4 points
    print(f"Running person {i}")
    x0 = [[x0[0],x0[1]] for x0 in person[:4]]
    objective = -gmm_model(params[i], xx, yy)            
    X_sample =  x0
    y_sample = [f_objective((xy[0],xy[1])) for xy in x0]
    current_best = np.max(y_sample)

    for k, point in enumerate(person[4:19]):
        lengthscale, output_variance, noise_variance = optimize_GP_hyperparams(X_sample, y_sample, 500, 5e-4, prior_mean, prior_std)
        mu, covariance = fit_predictive_GP(X_sample, y_sample, Xtest, lengthscale, output_variance, noise_variance)
        std = np.sqrt(np.diag(covariance))

        acquisition_values_pi = probability_of_improvement(current_best,  mu.flatten(), std, xi)
        acquisition_values_ei = expected_improvement(current_best,  mu.flatten(), std, xi)
        acquisition_values_gp_ucb = GP_UCB_original(mu.flatten(), std, kappa)

        beta = 5.0   # try different values later (or loop over betas)

        xh, yh = float(point[0]), float(point[1])
        idx_h = xy_to_flat_index(xh, yh)

        P_pi  = softmax(beta * acquisition_values_pi)
        P_ei  = softmax(beta * acquisition_values_ei)
        P_ucb = softmax(beta * acquisition_values_gp_ucb)

        logp_pi  = float(np.log(P_pi[idx_h]  + 1e-12))
        logp_ei  = float(np.log(P_ei[idx_h]  + 1e-12))
        logp_ucb = float(np.log(P_ucb[idx_h] + 1e-12))

        # BO-optimal points for each acquisition
        idx_PI  = np.argmax(acquisition_values_pi)
        idx_EI  = np.argmax(acquisition_values_ei)
        idx_UCB = np.argmax(acquisition_values_gp_ucb)

        xt_PI  = Xtest[idx_PI]
        xt_EI  = Xtest[idx_EI]
        xt_UCB = Xtest[idx_UCB]

        results.append({
            "person": int(i),
            "step": int(k),
            "beta": float(beta),

            "human_xy": [xh, yh],
            "human_z": float(point[2]),
            "idx_h": int(idx_h),

            "bo_PI_xy": xt_PI.tolist(),
            "bo_EI_xy": xt_EI.tolist(),
            "bo_UCB_xy": xt_UCB.tolist(),

            "logp_PI": logp_pi,
            "logp_EI": logp_ei,
            "logp_UCB": logp_ucb,

            "current_best": float(current_best),
        })

        X_sample.append(list(point[:2]))
        y_sample.append(point[2])
        current_best = np.max(y_sample)


Running person 0
Running person 1
Running person 2
Running person 3
Running person 4
Running person 5
Running person 6
Running person 7
Running person 8
Running person 9
Running person 10
Running person 11
Running person 12
Running person 13
Running person 14
Running person 15
Running person 16
Running person 17
Running person 18
Running person 19


### Save to json black magic

In [5]:
# Convert results so that everything is JSON-serializable
def make_json_serializable(results):
    serializable = []
    for entry in results:
        new_entry = {}
        for k, v in entry.items():
            if isinstance(v, np.ndarray):
                new_entry[k] = v.tolist()  # convert arrays to lists
            elif isinstance(v, (np.floating, np.float32, np.float64)):
                new_entry[k] = float(v)    # convert numpy floats to Python floats
            else:
                new_entry[k] = v
        serializable.append(new_entry)
    return serializable

serializable_results = make_json_serializable(results)

# Save to JSON
with open("results.json", "w") as f:
    json.dump(serializable_results, f, indent=2)